In [1]:
import os
from Bio import SeqIO
from collections import defaultdict

# Define input and output file paths
input_file = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/Covaris_shearing/TNSeq_13k_mutants_mapping_output_covaris_shearing/Tnseq_13k_mutants_covaris_shearing_blastquery.fasta'
mapped_file = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/Covaris_shearing/TNSeq_13k_mutants_mapping_output_covaris_shearing/Tnseq_13k_mutants_covaris_shearing_poolfile'
mapped_reads_output = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/Covaris_shearing/TNSeq_13k_mutants_mapping_output_covaris_shearing/digging_output/mapped_reads.fasta'
unmapped_reads_output = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/Covaris_shearing/TNSeq_13k_mutants_mapping_output_covaris_shearing/digging_output/unmapped_reads.fasta'
unmapped_reads_analysis_output = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencings/Sequencing_with_the_13k_mutants/for_analysis/Covaris_shearing/TNSeq_13k_mutants_mapping_output_covaris_shearing/digging_output/unmapped_reads_analysis.txt'

# Read mapped barcodes
mapped_barcodes = set()
with open(mapped_file, 'r') as mf:
    for line in mf:
        barcode = line.strip().split()[0]
        mapped_barcodes.add(barcode)

# Initialize read storage
mapped_reads = []
unmapped_reads = []

# Function to extract barcode
def extract_barcode(sequence):
    start_flank = "GTCTCT"
    end_flank = "CGTACG"
    start_index = sequence.find(start_flank)
    end_index = sequence.find(end_flank, start_index + len(start_flank))
    if start_index != -1 and end_index != -1:
        return sequence[start_index + len(start_flank):end_index]
    return None

# Process input fasta file
for record in SeqIO.parse(input_file, "fasta"):
    barcode = extract_barcode(str(record.seq))
    if barcode and barcode in mapped_barcodes:
        mapped_reads.append(record)
    else:
        unmapped_reads.append(record)

# Write mapped reads to output file
with open(mapped_reads_output, 'w') as mapped_out:
    SeqIO.write(mapped_reads, mapped_out, 'fasta')

# Write unmapped reads to output file
with open(unmapped_reads_output, 'w') as unmapped_out:
    SeqIO.write(unmapped_reads, unmapped_out, 'fasta')

# Analyze unmapped reads
read_lengths = [len(record.seq) for record in unmapped_reads]
short_reads = [length for length in read_lengths if length < 50]  # Assuming "too short" means less than 50bp

# Write analysis of unmapped reads
with open(unmapped_reads_analysis_output, 'w') as analysis_out:
    analysis_out.write(f"Total unmapped reads: {len(unmapped_reads)}\n")
    analysis_out.write(f"Reads shorter than 50bp: {len(short_reads)}\n")
    analysis_out.write(f"Percentage of short reads: {len(short_reads) / len(unmapped_reads) * 100:.2f}%\n")
    analysis_out.write(f"Read length distribution (first 100): {read_lengths[:100]}\n")

print("Script completed successfully.")


Script completed successfully.
